# Credit Risk Prediction under Imbalanced Dataset



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.utils import resample
from imblearn.under_sampling import ClusterCentroids
from collections import Counter
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from imblearn.under_sampling import TomekLinks
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from utils.opt import evaluate_with_gmean,eval_func, objective_xgb, objective_lgr
from utils.results import ResultsRegistry
from utils.sampling import apply_sampling
import yaml

In [2]:

registry = ResultsRegistry()

In [3]:

with open("utils/config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [4]:

READ_PATH = config["paths_model"]["read_file"]

In [5]:
df = pd.read_csv(READ_PATH)
df

,loan_amnt,funded_amnt_inv,term,installment,annual_inc,delinq_2yrs,open_acc,total_acc,tot_coll_amt,tot_cur_bal,...,income_to_installment,revolver_ratio,total_installment_amt,interest_rate,loan_to_income,loan_to_limit,open_acc_ratio,ever_delinq,emp_length_segment,emp_title_segment
0,16795.0,15094.0,36,32.66,228362.0,0.0,6.0,12.0,5.257495,30634.0,...,582.675036,2552.833333,1175.76,0.077896,0.073546,1.199986,0.500000,False,senior_exp,tech
1,1860.0,1494.0,60,2.17,273685.0,1.0,14.0,20.0,4.110874,158169.0,...,10510.176651,7908.450000,130.20,0.087149,0.006796,0.019137,0.700000,True,mid_exp,tech
2,39158.0,34683.0,36,76.14,165933.0,0.0,11.0,17.0,4.189655,66504.0,...,181.609535,3912.000000,2741.04,0.079031,0.235987,1.237845,0.647059,False,junior_exp,management
3,12284.0,11096.0,60,14.33,136818.0,0.0,7.0,16.0,3.931826,101207.0,...,795.638521,6325.437500,859.80,0.077487,0.089784,0.123025,0.437500,False,senior_exp,management
4,7265.0,6651.0,60,8.48,114755.0,0.0,13.0,15.0,4.488636,108914.0,...,1127.702437,7260.933333,508.80,0.076500,0.063309,0.105605,0.866667,False,junior_exp,professional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14579,9594.0,8471.0,36,18.66,195695.0,0.0,9.0,11.0,5.062595,132947.0,...,873.950518,12086.090909,671.76,0.079301,0.049025,0.189897,0.818182,False,senior_exp,tech
14580,14418.0,12611.0,36,28.04,85130.0,0.0,10.0,16.0,2.484907,99115.0,...,253.001664,6194.687500,1009.44,0.080044,0.169365,0.229729,0.625000,False,senior_exp,tech
14581,35854.0,34497.0,36,69.72,311044.0,0.0,12.0,16.0,3.332205,113572.0,...,371.777587,7098.250000,2509.92,0.072758,0.115270,0.459991,0.750000,False,senior_exp,professional
14582,24108.0,19647.0,36,46.88,288933.0,0.0,9.0,13.0,4.110874,136708.0,...,513.603882,10516.000000,1687.68,0.085900,0.083438,0.971157,0.692308,False,senior_exp,tech


In [6]:
y = df['default']
df.drop(columns=['default'], inplace=True)

categorical_cols = ['term', 'delinq_bucket', 'emp_length_segment', 'emp_title_segment']

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [7]:
X = df.copy()
y = y.astype(int)

## Majorty Undersampling (TomekLinks+ClusterCentroids) and Minorty Oversampling(ADASYN)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)



In [9]:
X_test

,loan_amnt,funded_amnt_inv,installment,annual_inc,delinq_2yrs,open_acc,total_acc,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,...,open_acc_ratio,ever_delinq,term_60,delinq_bucket_1,delinq_bucket_2,emp_length_segment_mid_exp,emp_length_segment_no_exp,emp_length_segment_senior_exp,emp_title_segment_professional,emp_title_segment_tech
14486,5270.0,4708.0,10.25,191180.0,0.0,13.0,18.0,5.945421,31231.0,61385.0,...,0.722222,False,False,False,False,False,False,True,False,False
14058,21547.0,20571.0,25.14,53969.0,0.0,20.0,23.0,4.094345,13674.0,50948.0,...,0.869565,False,True,False,False,True,False,False,False,True
1668,2667.0,2576.0,3.11,311924.0,0.0,8.0,12.0,4.634729,75617.0,104042.0,...,0.666667,False,True,False,False,False,True,False,False,True
9392,30991.0,29086.0,60.26,164472.0,0.0,9.0,15.0,5.056246,40624.0,4219.0,...,0.600000,False,False,False,False,True,False,False,False,False
10474,14546.0,14179.0,28.28,120163.0,0.0,9.0,15.0,5.942799,69072.0,17776.0,...,0.600000,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14373,13303.0,12737.0,25.87,283913.0,0.0,11.0,21.0,3.526361,45828.0,49009.0,...,0.523810,False,False,False,False,False,False,False,True,False
14435,33252.0,31272.0,64.66,287180.0,0.0,4.0,14.0,3.258097,33748.0,55632.0,...,0.285714,False,False,False,False,False,False,False,False,True
1395,6776.0,6243.0,13.18,205632.0,0.0,10.0,16.0,6.729824,160850.0,33173.0,...,0.625000,False,False,False,False,False,True,False,False,True
7426,8642.0,7362.0,16.80,273470.0,1.0,8.0,19.0,6.322565,153622.0,95034.0,...,0.421053,True,False,True,False,False,True,False,False,True


In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)
print("Train default oranı:", y_train.mean())
print("Test default oranı:", y_test.mean())

Train shape: (11667, 33)  Test shape: (2917, 33)
Train default oranı: 0.029999142881631954
Test default oranı: 0.030167980802194034


In [11]:
X_final, y_final= apply_sampling(
        X, 
        y, 
        use_tomek=True,
        use_cluster=True,
        use_random_under=False,
        use_over="adasyn",    
        verbose=True           
    )

📌 Original distribution: Counter({np.int64(0): 14146, np.int64(1): 438})

🔍 Applying TomekLinks...
➡️ After TomekLinks: Counter({np.int64(0): 13926, np.int64(1): 438})

🔻 Applying ClusterCentroids...
➡️ After ClusterCentroids: Counter({np.int64(0): 1314, np.int64(1): 438})

🔺 Applying ADASYN...
➡️ After ADASYN: Counter({np.int64(1): 1400, np.int64(0): 1314})

✅ Final distribution: Counter({np.int64(1): 1400, np.int64(0): 1314})


## xgboost ile modelleme

In [12]:
X_train, y_train=X_final, y_final

In [13]:
X_train_xbg, X_valid_xgb, y_train_xgb, y_valid_xgb = train_test_split(X_train, y_train, test_size=0.2, random_state=42,stratify=y_train)


In [14]:
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

In [15]:
# Optuna 
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective_xgb(trial,scale_pos_weight,X_train_xbg,X_valid_xgb,y_train_xgb,y_valid_xgb,X), n_trials=20)

print("En iyi hiperparametreler:", study.best_params)
print("En iyi F1 skoru:", study.best_value)
print("En iyi özellikler:", study.best_trial.user_attrs["selected_features"])

[I 2025-11-17 14:18:40,614] A new study created in memory with name: no-name-87be0406-f8f7-4c07-a5be-a6fb9f5e2467
[I 2025-11-17 14:18:41,726] Trial 0 finished with value: 0.9579990009508998 and parameters: {'n_estimators': 305, 'max_depth': 4, 'learning_rate': 0.15074832350305534, 'subsample': 0.6577979164125122, 'colsample_bytree': 0.8617891143643372, 'gamma': 3.0184186020659807, 'min_child_weight': 7, 'reg_alpha': 0.331221290504381, 'reg_lambda': 7.678439499257301, 'scale_pos_weight': 0.7323557977884512, 'feature_0': 0, 'feature_1': 0, 'feature_2': 1, 'feature_3': 0, 'feature_4': 1, 'feature_5': 1, 'feature_6': 0, 'feature_7': 0, 'feature_8': 1, 'feature_9': 0, 'feature_10': 0, 'feature_11': 0, 'feature_12': 1, 'feature_13': 1, 'feature_14': 0, 'feature_15': 1, 'feature_16': 0, 'feature_17': 0, 'feature_18': 0, 'feature_19': 1, 'feature_20': 0, 'feature_21': 0, 'feature_22': 1, 'feature_23': 0, 'feature_24': 1, 'feature_25': 0, 'feature_26': 1, 'feature_27': 1, 'feature_28': 1, 'feat

En iyi hiperparametreler: {'n_estimators': 181, 'max_depth': 5, 'learning_rate': 0.20913814966131222, 'subsample': 0.6022524138477343, 'colsample_bytree': 0.7123774509808223, 'gamma': 2.089691798868744, 'min_child_weight': 9, 'reg_alpha': 7.079039825679987, 'reg_lambda': 4.8883794892914, 'scale_pos_weight': 1.1619007551304381, 'feature_0': 0, 'feature_1': 1, 'feature_2': 1, 'feature_3': 0, 'feature_4': 1, 'feature_5': 0, 'feature_6': 0, 'feature_7': 1, 'feature_8': 0, 'feature_9': 1, 'feature_10': 1, 'feature_11': 0, 'feature_12': 0, 'feature_13': 0, 'feature_14': 1, 'feature_15': 1, 'feature_16': 1, 'feature_17': 1, 'feature_18': 0, 'feature_19': 0, 'feature_20': 1, 'feature_21': 1, 'feature_22': 0, 'feature_23': 1, 'feature_24': 0, 'feature_25': 1, 'feature_26': 1, 'feature_27': 0, 'feature_28': 1, 'feature_29': 0, 'feature_30': 1, 'feature_31': 0, 'feature_32': 1}
En iyi F1 skoru: 0.9777536360107338
En iyi özellikler: ['funded_amnt_inv', 'installment', 'delinq_2yrs', 'tot_coll_amt',

In [16]:
best_mask = [study.best_trial.params[f"feature_{i}"] for i in range(X.shape[1])]
best_features = [col for col, m in zip(X.columns, best_mask) if m == 1]
print("Seçilen özellikler:", best_features)

Seçilen özellikler: ['funded_amnt_inv', 'installment', 'delinq_2yrs', 'tot_coll_amt', 'total_rev_hi_lim', 'emp_length_num', 'utilization_ratio', 'debt_to_income', 'installment_to_income', 'income_to_installment', 'interest_rate', 'loan_to_income', 'open_acc_ratio', 'term_60', 'delinq_bucket_1', 'emp_length_segment_mid_exp', 'emp_length_segment_senior_exp', 'emp_title_segment_tech']


In [17]:
X_train_best = X_train_xbg[best_features]
X_test_best = X_test[best_features]
xgb_best = XGBClassifier(**{k:v for k,v in study.best_params.items() if 'feature_' not in k})
xgb_best.fit(X_train_best, y_train_xgb)

# Threshold optimizasyonu
y_proba_xgb = xgb_best.predict_proba(X_test_best)[:, 1]


best_th_xgb, metrics_xgb, y_pred_xgb = evaluate_with_gmean(y_test, y_proba_xgb)


Best threshold (based on G-Mean): 0.985
Sensitivity: 0.557, Specificity: 0.509, G-Mean: 0.533


In [18]:

y_pred_thresh = (y_proba_xgb >= best_th_xgb).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_thresh)
cm

array([[1441, 1388],
       [  39,   49]])

In [19]:
eval_func(y_test, y_pred_thresh)

{'TN': np.int64(1441),
 'FP': np.int64(1388),
 'FN': np.int64(39),
 'TP': np.int64(49),
 'sensitivity': np.float64(0.5568181818181818),
 'specificity': np.float64(0.5093672675857194),
 'g_mean': np.float64(0.5325645085947575)}

In [20]:
registry.add_model(
    model_name="xgb_optuna_gmean",
    best_threshold=best_th_xgb,
    metrics=metrics_xgb,
    y_true=y_test,
    y_pred=y_pred_thresh,
    y_proba=y_proba_xgb,
    model=xgb_best,
    score=metrics_xgb["g_mean"]
)

## Logistic Reg 

In [21]:
X_test_scaled=scaler.transform(X_test)

In [22]:
X_train_scaled = scaler.fit_transform(X_train)


In [23]:
X_train_lgr, X_valid_lgr, y_train_lgr, y_valid_lgr = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42,stratify=y_train)



In [24]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective_lgr(trial,X_train_lgr,y_train_lgr,X_valid_lgr,y_valid_lgr), n_trials=20)


study.optimize(
    lambda trial: objective_lgr(
        trial,
        X_train_lgr,
        y_train_lgr,
        X_valid_lgr,
        y_valid_lgr
    ),
    n_trials=20
)

print("Best trial params:")
print(study.best_trial.params)
print("Best value (G-Mean):", study.best_value)



[I 2025-11-17 14:18:51,027] A new study created in memory with name: no-name-f6e679f9-1b18-4844-abf8-4c7190f4ac77
[I 2025-11-17 14:18:51,045] Trial 0 finished with value: 0.9849905631832326 and parameters: {'C': 0.02382535517940148, 'penalty': 'l2', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.9849905631832326.
[I 2025-11-17 14:18:51,062] Trial 1 finished with value: 0.98467342253634 and parameters: {'C': 7.266033568980621, 'penalty': 'l2', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.9849905631832326.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-11-17 14:18:51,139] Trial 2 finished with value: 0.9829135043779482 and parameters: {'C': 0.053542819702219635, 'penalty': 'none', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.9849905631832326.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: S

Best trial params:
{'C': 0.10032136865542074, 'penalty': 'l2', 'solver': 'saga'}
Best value (G-Mean): 0.9867605386686007


In [25]:
best_params = study.best_trial.params

# Modeli oluştur
lr_best = LogisticRegression(
    C=best_params['C'],
    penalty=best_params['penalty'],
    solver=best_params['solver'],
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)
lr_best.fit(X_train_scaled, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.10032136865542074
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'saga'
,max_iter,1000
,multi_class,'deprecated'


In [26]:
lr_probs = lr_best.predict_proba(X_test_scaled)[:, 1]


best_th_lgr, metrics_lgr, y_pred_lgr = evaluate_with_gmean(y_test, lr_probs)



Best threshold (based on G-Mean): 0.657
Sensitivity: 0.580, Specificity: 0.505, G-Mean: 0.541


In [27]:
y_pred_thresh_lr = (lr_probs >= best_th_lgr).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_thresh_lr)
cm

array([[1428, 1401],
       [  37,   51]])

In [28]:
eval_func(y_test, y_pred_thresh_lr)

{'TN': np.int64(1428),
 'FP': np.int64(1401),
 'FN': np.int64(37),
 'TP': np.int64(51),
 'sensitivity': np.float64(0.5795454545454546),
 'specificity': np.float64(0.5047720042417816),
 'g_mean': np.float64(0.540868117603657)}

In [29]:
registry.add_model(
    model_name="logreg_optuna_gmean",
    best_threshold=best_th_lgr,
    metrics=metrics_lgr,
    y_true=y_test,
    y_pred=y_pred_thresh_lr,
    y_proba=lr_probs,
    model=lr_best,
    score=metrics_lgr["g_mean"]
)

## Logistic Reg + RandomUnderSampler

In [30]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_resampled, y_resampled= apply_sampling(
        X_train_scaled, 
        y_train, 
        use_tomek=False,
        use_cluster=False,
        use_random_under=True,
        use_over=None,    
        verbose=True           
    )


📌 Original distribution: Counter({np.int64(0): 9901, np.int64(1): 307})

🔻 Applying RandomUnderSampler...
➡️ After RandomUnderSampler: Counter({np.int64(0): 307, np.int64(1): 307})

✅ Final distribution: Counter({np.int64(0): 307, np.int64(1): 307})


In [31]:

X_train_lgr, X_valid_lgr, y_train_lgr, y_valid_lgr = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42,stratify=y_resampled)

# Optuna çalıştır
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective_lgr(trial,X_train_lgr,y_train_lgr,X_valid_lgr,y_valid_lgr), n_trials=20)

print("Best trial:")
print(study.best_trial.params)

[I 2025-11-17 14:19:09,736] A new study created in memory with name: no-name-c116fabd-5d7d-4066-b645-f24dbdd951ef
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-11-17 14:19:10,014] Trial 0 finished with value: 0.5111139212618532 and parameters: {'C': 5.444317139655686, 'penalty': 'none', 'solver': 'saga'}. Best is trial 0 with value: 0.5111139212618532.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2025-11-17 14:19:10,293] Trial 1 finished with value: 0.5111139212618532 and parameters: {'C': 0.0018364826752794586, 'penalty': 'none', 'solver': 'saga'}. Best is trial 0 with value: 0.5111139212618532.
c:\conda_envs\kredi_proje\Lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will

Best trial:
{'C': 0.1286578117071329, 'penalty': 'l2', 'solver': 'lbfgs'}


In [32]:
best_params = study.best_trial.params

# Modeli oluştur
lr_best2 = LogisticRegression(
    C=best_params['C'],
    penalty=best_params['penalty'],
    solver=best_params['solver'],
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)
lr_best2.fit(X_train_lgr, y_train_lgr)


lr_probs = lr_best2.predict_proba(X_test_scaled)[:, 1]

# G-Mean'e göre en iyi threshold ve metrikler

best_th_rf, metrics_rf, y_pred_rf = evaluate_with_gmean(y_test, lr_probs)



Best threshold (based on G-Mean): 0.506
Sensitivity: 0.511, Specificity: 0.529, G-Mean: 0.520


In [33]:
y_pred_thresh_rf = (lr_probs >= best_th_rf).astype(int)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_thresh_rf)

In [34]:
registry.add_model(
    model_name="rf_baseline",
    best_threshold=best_th_rf,
    metrics=metrics_rf,
    y_true=y_test,
    y_pred=y_pred_thresh_rf,
    y_proba=lr_probs,
    model=lr_best2,
    score=metrics_rf["g_mean"]
)


In [35]:
confusion_matrix(y_test, y_pred_thresh_rf)

array([[2246, 1999],
       [  64,   67]])

In [36]:
print(registry.results.keys())
for name, res in registry.results.items():
    print("Model:", name)
    print("  threshold:", res.best_threshold)
    print("  metrics:", res.metrics)


dict_keys(['xgb_optuna_gmean', 'logreg_optuna_gmean', 'rf_baseline'])
Model: xgb_optuna_gmean
  threshold: 0.9850870370864868
  metrics: {'tn': np.int64(1441), 'fp': np.int64(1388), 'fn': np.int64(39), 'tp': np.int64(49), 'sensitivity': np.float64(0.5568181818181818), 'specificity': np.float64(0.5093672675857194), 'g_mean': np.float64(0.5325645085947575)}
Model: logreg_optuna_gmean
  threshold: 0.6569086785159414
  metrics: {'tn': np.int64(1428), 'fp': np.int64(1401), 'fn': np.int64(37), 'tp': np.int64(51), 'sensitivity': np.float64(0.5795454545454546), 'specificity': np.float64(0.5047720042417816), 'g_mean': np.float64(0.540868117603657)}
Model: rf_baseline
  threshold: 0.5062409047928702
  metrics: {'tn': np.int64(2246), 'fp': np.int64(1999), 'fn': np.int64(64), 'tp': np.int64(67), 'sensitivity': np.float64(0.5114503816793893), 'specificity': np.float64(0.529093050647821), 'g_mean': np.float64(0.5201969268438065)}


In [37]:
run_dir = registry.save_all(base_dir="results", save_models=True)
print(run_dir)

results\run_20251117_141829
